In [2]:
import ast
from bs4 import BeautifulSoup
import csv
import json
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import re
import requests
from scipy import stats
import seaborn as sns
import sqlite3
from sqlite3 import Error
import time
import warnings
warnings.filterwarnings("ignore")

In [3]:
def product_url_finder(main_url, pageCount):
    product_url_list = []
    url = None
    for i in range(1, pageCount+1):
        number = str(i)
        url = main_url + "?pageSize=10&currentPage=" + number
        print("Scraping: " + url)
        r = requests.get(url)
        if r.status_code == 200:
            c = r.content
            soup = BeautifulSoup(c, 'html.parser')
            product_containers = soup.find_all("div", class_="css-1aj5qq4")
            for product_container in product_containers:
                product_loads = product_container.find_all("div", class_="css-dkxsdo")
            for product_load in product_loads:
                base_url = "https://www.sephora.com"
                product_links = product_load.find_all("a", class_="css-ix8km1")
                for product_link in product_links:
                    product_url = product_link["href"].replace(" ", "%20")
                    complete_url = base_url + product_url
                    product_url_list.append(complete_url)
        else:
            print(r.status_code)
    return product_url_list

In [410]:
product_list = product_url_finder("https://www.sephora.com/shop/moisturizing-cream-oils-mists", 84)

Scraping: https://www.sephora.com/shop/moisturizing-cream-oils-mists?pageSize=10&currentPage=1
Scraping: https://www.sephora.com/shop/moisturizing-cream-oils-mists?pageSize=10&currentPage=2
Scraping: https://www.sephora.com/shop/moisturizing-cream-oils-mists?pageSize=10&currentPage=3
Scraping: https://www.sephora.com/shop/moisturizing-cream-oils-mists?pageSize=10&currentPage=4
Scraping: https://www.sephora.com/shop/moisturizing-cream-oils-mists?pageSize=10&currentPage=5
Scraping: https://www.sephora.com/shop/moisturizing-cream-oils-mists?pageSize=10&currentPage=6
Scraping: https://www.sephora.com/shop/moisturizing-cream-oils-mists?pageSize=10&currentPage=7
Scraping: https://www.sephora.com/shop/moisturizing-cream-oils-mists?pageSize=10&currentPage=8
Scraping: https://www.sephora.com/shop/moisturizing-cream-oils-mists?pageSize=10&currentPage=9
Scraping: https://www.sephora.com/shop/moisturizing-cream-oils-mists?pageSize=10&currentPage=10
Scraping: https://www.sephora.com/shop/moisturizi

In [411]:
len(product_list)

837

In [412]:
product_series = pd.Series(product_list)

In [413]:
product_series.to_csv(r"/Users/florencialeoni/Documents/flatironDS/Mod2/i_feel_pretty/product_urls.csv")

In [43]:
def product_info_finder(url_list):
    products = {"brand": [], "name": [], "price": [], "n_of_reviews": [], 
               "n_of_loves": [], "review_score": [], "size": [], "clean_product": [], 
                "category": []}

    for product_url in url_list:
        print("Product URL: " + product_url)
        r = requests.get(product_url)
        if r.status_code == 200:
            c = r.content
            soup = BeautifulSoup(c, 'html.parser')
            # Get a hold of the main product information container.
            main_container = soup.find("main", class_="css-1o80i28")
            if main_container is not None:
                
                # Get the product category.
                product_category = main_container.find("h1", class_="css-bnsadm")
                if product_category is None:
                    products["category"].append("None")
                else:
                    products["category"].append(product_category.text)

                # Get brand.
                product_brand = main_container.find("span", class_="css-euydo4").text
                products["brand"].append(product_brand)

                # Get product name.
                product_name = main_container.find("span", class_="css-0").text
                products["name"].append(product_name)

                # Get product price.
                product_price = main_container.find("div", class_="css-14hdny6").text
                products["price"].append(product_price.replace("$",""))

                # Get the product size. This is tricky, since it might be a single option,
                # multiple, or it might not have a size at all.

                product_size = main_container.find("div", class_="css-1qf1va5").span

                if product_size is not None:
                    products["size"].append(product_size.text.replace("SIZE ", "").split("/")[0])
                else:
                    product_size = main_container.find("button", class_="css-cl742e")
                    if product_size is not None:
                        products["size"].append(product_size.text.split("/")[0])
                    else:                
                        products["size"].append("None")

                # The the number of product loves.
                product_n_loves = main_container.find("div", class_="css-th1w6d").span.text
                products["n_of_loves"].append(product_n_loves.replace(" loves",""))

                # Find out if it's labeled "Clean at Sephora".
                product_clean = main_container.find_all(text="Clean at Sephora")
                if product_clean:
                    products["clean_product"].append(1)
                else:
                    products["clean_product"].append(0)

                # Get the number of reviews.
                product_n_reviews = main_container.find("span", class_="css-2rg6q7").text.replace(" reviews","").replace(" review", "").split("K")
                if len(product_n_reviews) > 1:
                    product_n_reviews = int(product_n_reviews[0]) * 1000
                else:
                    product_n_reviews = int(product_n_reviews[0])

                products["n_of_reviews"].append(product_n_reviews)

                # Get the score.
                product_score = 0.05*float(main_container.find("div", class_="css-1pjru6n")["style"].replace("width:", "").replace("%", ""))
                products["review_score"].append(product_score)           

        else:
            print(r.status_code)
    return products

In [424]:
moisturizer_dict = product_info_finder(product_list)

Product URL: https://www.sephora.com/product/protini-tm-polypeptide-cream-P427421?icid2=products%20grid:p427421:product
Product URL: https://www.sephora.com/product/creme-de-la-mer-moisturizing-cream-P416341?icid2=products%20grid:p416341:product
Product URL: https://www.sephora.com/product/your-skin-but-better-cc-cream-spf-50-P411885?icid2=products%20grid:p411885:product
Product URL: https://www.sephora.com/product/the-water-cream-P418218?icid2=products%20grid:p418218:product
Product URL: https://www.sephora.com/product/facial-treatment-essence-P375849?icid2=products%20grid:p375849:product
Product URL: https://www.sephora.com/product/lala-retro-whipped-cream-P408645?icid2=products%20grid:p408645:product
Product URL: https://www.sephora.com/product/the-dewy-skin-cream-P441101?icid2=products%20grid:p441101:product
Product URL: https://www.sephora.com/product/virgin-marula-tm-luxury-facial-oil-P392245?icid2=products%20grid:p392245:product
Product URL: https://www.sephora.com/product/the-o

Product URL: https://www.sephora.com/product/glowstarter-mega-illuminating-moisturizer-P408739?icid2=products%20grid:p408739:product
Product URL: https://www.sephora.com/product/water-bank-moisture-cream-P420655?icid2=products%20grid:p420655:product
Product URL: https://www.sephora.com/product/tidal-brightening-enzyme-water-cream-P406938?icid2=products%20grid:p406938:product
Product URL: https://www.sephora.com/product/kale-spinach-hyaluronic-acid-age-prevention-cream-P411388?icid2=products%20grid:p411388:product
Product URL: https://www.sephora.com/product/watermelon-glow-sleeping-mask-P420160?icid2=products%20grid:p420160:product
Product URL: https://www.sephora.com/product/secret-sauce-clinically-advanced-miraculous-anti-aging-moisturizer-P421958?icid2=products%20grid:p421958:product
Product URL: https://www.sephora.com/product/goodnight-glow-retin-alt-sleeping-creme-P434546?icid2=products%20grid:p434546:product
Product URL: https://www.sephora.com/product/emerald-cannabis-sativa-he

Product URL: https://www.sephora.com/product/aqua-bomb-sleeping-mask-P433443?icid2=products%20grid:p433443:product
Product URL: https://www.sephora.com/product/dr-andrew-weil-for-origins-mega-bright-spf-30-skin-tone-correcting-oil-free-moisturizer-P385253?icid2=products%20grid:p385253:product
Product URL: https://www.sephora.com/product/the-renewal-oil-P417241?icid2=products%20grid:p417241:product
Product URL: https://www.sephora.com/product/peat-miracle-revital-cream-P412440?icid2=products%20grid:p412440:product
Product URL: https://www.sephora.com/product/the-moisturizing-cool-gel-cream-P429637?icid2=products%20grid:p429637:product
Product URL: https://www.sephora.com/product/facial-spray-with-aloe-herbs-rosewater-P440482?icid2=products%20grid:p440482:product
Product URL: https://www.sephora.com/product/hydro-dynamic-ultimate-moisture-P268700?icid2=products%20grid:p268700:product
Product URL: https://www.sephora.com/product/ginzing-spf-40-energy-boosting-tinted-moisturizer-P420966?ic

Product URL: https://www.sephora.com/product/black-label-detox-bb-beauty-balm-P374568?icid2=products%20grid:p374568:product
Product URL: https://www.sephora.com/product/phoenix-cell-regenerating-facial-oil-P400204?icid2=products%20grid:p400204:product
Product URL: https://www.sephora.com/product/all-day-hydrator-hydrate-glow-P438617?icid2=products%20grid:p438617:product
Product URL: https://www.sephora.com/product/photo-finish-primerizer-P421232?icid2=products%20grid:p421232:product
Product URL: https://www.sephora.com/product/vitamin-enriched-face-base-priming-moisturizer-mini-P441804?icid2=products%20grid:p441804:product
Product URL: https://www.sephora.com/product/colored-clay-cc-undereye-corrector-P387175?icid2=products%20grid:p387175:product
Product URL: https://www.sephora.com/product/hyaluronic-marine-oil-free-moisture-cushion-P416755?icid2=products%20grid:p416755:product
Product URL: https://www.sephora.com/product/photo-finish-primer-oil-P404235?icid2=products%20grid:p404235:p

Product URL: https://www.sephora.com/product/volcanic-elixir-P430714?icid2=products%20grid:p430714:product
Product URL: https://www.sephora.com/product/100-squalane-oil-P416564?icid2=products%20grid:p416564:product
Product URL: https://www.sephora.com/product/bb-cushion-pore-blur-P420658?icid2=products%20grid:p420658:product
Product URL: https://www.sephora.com/product/5-in-1-face-cream-spf-30-P263006?icid2=products%20grid:p263006:product
Product URL: https://www.sephora.com/product/superdefense-spf-20-age-defense-moisturizer-very-dry-to-dry-combination-P223707?icid2=products%20grid:p223707:product
Product URL: https://www.sephora.com/product/benefiance-wrinkleresist24-day-emulsion-broad-spectrum-spf-18-P173614?icid2=products%20grid:p173614:product
Product URL: https://www.sephora.com/product/dramatically-different-hydrating-jelly-P432241?icid2=products%20grid:p432241:product
Product URL: https://www.sephora.com/product/100-pure-argan-oil-light-P380805?icid2=products%20grid:p380805:pro

...

Product URL: https://www.sephora.com/product/hungarian-thermal-water-mineral-rich-moisturizer-P427637?icid2=products%20grid:p427637:product
Product URL: https://www.sephora.com/product/balancing-moisturizer-broad-spectrum-spf-15-pa-P379902?icid2=products%20grid:p379902:product
Product URL: https://www.sephora.com/product/future-solution-lx-total-regenerating-cream-P423169?icid2=products%20grid:p423169:product
Product URL: https://www.sephora.com/product/vital-treatment-essence-for-hydrating-P408268?icid2=products%20grid:p408268:product
Product URL: https://www.sephora.com/product/hydrating-moisturizer-for-dry-skin-P423740?icid2=products%20grid:p423740:product
Product URL: https://www.sephora.com/product/repairwear-sculpting-night-cream-P392241?icid2=products%20grid:p392241:product
Product URL: https://www.sephora.com/product/pureness-moisturizing-gel-cream-P173645?icid2=products%20grid:p173645:product
Product URL: https://www.sephora.com/product/rejuvenating-lift-for-neck-decollete-P29

Product URL: https://www.sephora.com/product/c-collagen-perfect-skin-set-refresh-mist-P411361?icid2=products%20grid:p411361:product
Product URL: https://www.sephora.com/product/the-radiance-brightening-vitamin-e-rosehip-seed-facial-oil-P438629?icid2=products%20grid:p438629:product
Product URL: https://www.sephora.com/product/dry-erase-ultra-calming-face-cream-P397665?icid2=products%20grid:p397665:product
Product URL: https://www.sephora.com/product/make-a-difference-trade-plus-43-rejuvenating-treatment-P376633?icid2=products%20grid:p376633:product
Product URL: https://www.sephora.com/product/no-4-vanilla-silk-hydrating-lotion-P428643?icid2=products%20grid:p428643:product
Product URL: https://www.sephora.com/product/complexion-perfecting-hydrator-P440063?icid2=products%20grid:p440063:product
Product URL: https://www.sephora.com/product/oil-control-mattifying-mist-P421949?icid2=products%20grid:p421949:product
Product URL: https://www.sephora.com/product/and-coco-bliss-intensive-repair-P4

Product URL: https://www.sephora.com/product/cactus-flower-tibetan-ginseng-hydrating-mist-P426825?icid2=products%20grid:p426825:product
Product URL: https://www.sephora.com/product/multi-miracle-glow-P433524?icid2=products%20grid:p433524:product
Product URL: https://www.sephora.com/product/3-in-1-cleansing-oil-P416596?icid2=products%20grid:p416596:product
Product URL: https://www.sephora.com/product/white-lucent-day-emulsion-broad-spectrum-spf-23-P443377?icid2=products%20grid:p443377:product
Product URL: https://www.sephora.com/product/advanced-time-zone-age-reversing-line-wrinkle-hydrating-gel-oil-free-P380006?icid2=products%20grid:p380006:product
Product URL: https://www.sephora.com/product/yuza-sorbet-night-reinforced-nighttime-emulsion-P406940?icid2=products%20grid:p406940:product
Product URL: https://www.sephora.com/product/pure-nurture-facial-water-P408233?icid2=products%20grid:p408233:product
Product URL: https://www.sephora.com/product/glow-creme-illuminating-face-cream-P417244

Product URL: https://www.sephora.com/product/cosmic-maracuja-concentrated-face-balm-P433154?icid2=products%20grid:p433154:product
Product URL: https://www.sephora.com/product/calming-lavender-mist-P423731?icid2=products%20grid:p423731:product
Product URL: https://www.sephora.com/product/oil-control-daily-moisturizer-P440288?icid2=products%20grid:p440288:product
Product URL: https://www.sephora.com/product/even-better-skin-tone-correcting-lotion-broad-spectrum-spf-20-P309226?icid2=products%20grid:p309226:product
Product URL: https://www.sephora.com/product/blu-mediterraneo-italian-resort-moisturizing-face-lotion-P408979?icid2=products%20grid:p408979:product
Product URL: https://www.sephora.com/product/aqua-reotier-ultra-thirst-quenching-cream-moisturizer-P429938?icid2=products%20grid:p429938:product
Product URL: https://www.sephora.com/product/nuddy-ultra-hydrating-repair-cr-me-P430834?icid2=products%20grid:p430834:product
Product URL: https://www.sephora.com/product/time-retreat-radian

Product URL: https://www.sephora.com/product/multi-biotic-moisturizer-P443832?icid2=products%20grid:p443832:product
Product URL: https://www.sephora.com/product/skin-soothing-hydrating-lotion-P443368?icid2=products%20grid:p443368:product
Product URL: https://www.sephora.com/product/squalane-oil-P443839?icid2=products%20grid:p443839:product
Product URL: https://www.sephora.com/product/turmeric-moisturizer-P443835?icid2=products%20grid:p443835:product


In [425]:
moisturizers_df = pd.DataFrame.from_dict(moisturizer_dict)

In [426]:
moisturizers_df.to_csv(r"/Users/florencialeoni/Documents/flatironDS/Mod2/i_feel_pretty/moisturizers.csv")

In [66]:
skincare_df = pd.read_excel("/Users/florencialeoni/code/i_feel_pretty/data/moisturizers.xls")

In [16]:
treatments_url = product_url_finder("https://www.sephora.com/shop/facial-treatments", 58)

Scraping: https://www.sephora.com/shop/facial-treatments?pageSize=10&currentPage=1
Scraping: https://www.sephora.com/shop/facial-treatments?pageSize=10&currentPage=2
Scraping: https://www.sephora.com/shop/facial-treatments?pageSize=10&currentPage=3
Scraping: https://www.sephora.com/shop/facial-treatments?pageSize=10&currentPage=4
Scraping: https://www.sephora.com/shop/facial-treatments?pageSize=10&currentPage=5
Scraping: https://www.sephora.com/shop/facial-treatments?pageSize=10&currentPage=6
Scraping: https://www.sephora.com/shop/facial-treatments?pageSize=10&currentPage=7
Scraping: https://www.sephora.com/shop/facial-treatments?pageSize=10&currentPage=8
Scraping: https://www.sephora.com/shop/facial-treatments?pageSize=10&currentPage=9
Scraping: https://www.sephora.com/shop/facial-treatments?pageSize=10&currentPage=10
Scraping: https://www.sephora.com/shop/facial-treatments?pageSize=10&currentPage=11
Scraping: https://www.sephora.com/shop/facial-treatments?pageSize=10&currentPage=12
S

In [17]:
treatment_series = pd.Series(treatments_url)

In [18]:
treatment_series.to_csv(r"/Users/florencialeoni/code/i_feel_pretty/data/treatment_urls.csv")

In [19]:
treatment_dict = product_info_finder(treatments_url)

Product URL: https://www.sephora.com/product/double-serum-complete-age-control-concentrate-P421235?icid2=products%20grid:p421235:product
Product URL: https://www.sephora.com/product/c-firma-day-serum-P400259?icid2=products%20grid:p400259:product
Product URL: https://www.sephora.com/product/t-l-c-sukari-babyfacial-P415777?icid2=products%20grid:p415777:product
Product URL: https://www.sephora.com/product/t-l-c-framboos-tm-glycolic-night-serum-P392246?icid2=products%20grid:p392246:product
Product URL: https://www.sephora.com/product/good-genes-all-in-one-lactic-acid-treatment-P309308?icid2=products%20grid:p309308:product
Product URL: https://www.sephora.com/product/facial-treatment-essence-P375849?icid2=products%20grid:p375849:product
Product URL: https://www.sephora.com/product/advanced-night-repair-synchronized-recovery-complex-ii-duo-P405491?icid2=products%20grid:p405491:product
Product URL: https://www.sephora.com/product/truth-serum-P42343?icid2=products%20grid:p42343:product
Product

Product URL: https://www.sephora.com/product/the-regenerating-serum-P417601?icid2=products%20grid:p417601:product
Product URL: https://www.sephora.com/product/sunless-tan-anti-aging-face-serum-P418221?icid2=products%20grid:p418221:product
Product URL: https://www.sephora.com/product/potent-c-power-serum-P428100?icid2=products%20grid:p428100:product
Product URL: https://www.sephora.com/product/retinol-youth-renewal-serum-P411365?icid2=products%20grid:p411365:product
Product URL: https://www.sephora.com/product/peace-out-pores-P428250?icid2=products%20grid:p428250:product
Product URL: https://www.sephora.com/product/eradikate-acne-treatment-P232903?icid2=products%20grid:p232903:product
Product URL: https://www.sephora.com/product/ascorbic-acid-8-alpha-arbutin-2-P442754?icid2=products%20grid:p442754:product
Product URL: https://www.sephora.com/product/blue-tansy-aha-bha-resurfacing-clarity-mask-P400207?icid2=products%20grid:p400207:product
Product URL: https://www.sephora.com/product/vino

Product URL: https://www.sephora.com/product/blackhead-solutions-self-heating-blackhead-extractor-P432243?icid2=products%20grid:p432243:product
Product URL: https://www.sephora.com/product/glow-cycle-retin-alt-power-serum-P434545?icid2=products%20grid:p434545:product
Product URL: https://www.sephora.com/product/rainforest-sea-deep-sea-collagen-super-serum-P440814?icid2=products%20grid:p440814:product
Product URL: https://www.sephora.com/product/no-foundation-foundation-serum-spf-30-P386465?icid2=products%20grid:p386465:product
Product URL: https://www.sephora.com/product/water-drench-hyaluronic-cloud-serum-P222818?icid2=products%20grid:p222818:product
Product URL: https://www.sephora.com/product/vinopure-natural-salicylic-acid-pore-minimizing-toner-P433625?icid2=products%20grid:p433625:product
Product URL: https://www.sephora.com/product/time-release-acne-cleanser-P254720?icid2=products%20grid:p254720:product
Product URL: https://www.sephora.com/product/b-oil-P442757?icid2=products%20g

Product URL: https://www.sephora.com/product/capture-youth-plump-filler-age-delay-plumping-serum-P427732?icid2=products%20grid:p427732:product
Product URL: https://www.sephora.com/product/premier-cru-serum-P427509?icid2=products%20grid:p427509:product
Product URL: https://www.sephora.com/product/focuspot-line-wrinkle-micro-tip-patch-P442856?icid2=products%20grid:p442856:product
Product URL: https://www.sephora.com/product/pure-skin-clarifying-dietary-supplement-P7365?icid2=products%20grid:p7365:product
Product URL: https://www.sephora.com/product/future-solution-lx-intensive-firming-contour-serum-P423166?icid2=products%20grid:p423166:product
Product URL: https://www.sephora.com/product/breakout-fighters-P436363?icid2=products%20grid:p436363:product
Product URL: https://www.sephora.com/product/prep-n-glow-cloths-P414661?icid2=products%20grid:p414661:product
Product URL: https://www.sephora.com/product/neuropeptide-facial-conformer-contouring-treatment-P211332?icid2=products%20grid:p2113

Product URL: https://www.sephora.com/product/age-arrest-anti-wrinkle-cream-P382356?icid2=products%20grid:p382356:product
Product URL: https://www.sephora.com/product/my-daily-dose-custom-blended-serum-set-P394357?icid2=products%20grid:p394357:product
Product URL: https://www.sephora.com/product/acne-clearing-wash-P410883?icid2=products%20grid:p410883:product
Product URL: https://www.sephora.com/product/dr-andrew-weil-for-origins-tm-mega-mushroom-relief-resilience-advanced-face-serum-P429245?icid2=products%20grid:p429245:product
Product URL: https://www.sephora.com/product/caviar-lime-acid-peel-P421741?icid2=products%20grid:p421741:product
Product URL: https://www.sephora.com/product/beauty-shroom-tm-plumping-jelly-serum-P432831?icid2=products%20grid:p432831:product
Product URL: https://www.sephora.com/product/hyalo-plasma-hyaluronic-intensive-moisturizer-P388974?icid2=products%20grid:p388974:product
Product URL: https://www.sephora.com/product/omg-omega-great-supplements-P386760?icid2=

Product URL: https://www.sephora.com/product/firming-serum-with-energy-peptides-P437240?icid2=products%20grid:p437240:product
Product URL: https://www.sephora.com/product/fresh-pressed-7-day-system-with-pure-vitamin-c-P417617?icid2=products%20grid:p417617:product
Product URL: https://www.sephora.com/product/micro-blur-skin-perfector-P422499?icid2=products%20grid:p422499:product
Product URL: https://www.sephora.com/product/high-potency-amine-complex-face-lift-P243237?icid2=products%20grid:p243237:product
Product URL: https://www.sephora.com/product/3-in-1-extractor-P231607?icid2=products%20grid:p231607:product
Product URL: https://www.sephora.com/product/fix-line-smoothing-serum-P440013?icid2=products%20grid:p440013:product
Product URL: https://www.sephora.com/product/dr-andrew-weil-for-origins-mega-bright-dark-spot-correcting-serum-P408237?icid2=products%20grid:p408237:product
Product URL: https://www.sephora.com/product/proactiv-solution-3-step-acne-treatment-system-90-day-size-P44065

Product URL: https://www.sephora.com/product/egf-activating-serum-P441051?icid2=products%20grid:p441051:product
Product URL: https://www.sephora.com/product/blackhead-dissolving-gel-P440059?icid2=products%20grid:p440059:product
Product URL: https://www.sephora.com/product/breakout-control-targeted-acne-spot-treatment-P422641?icid2=products%20grid:p422641:product
Product URL: https://www.sephora.com/product/renewed-hope-in-jar-peeling-mousse-instant-glow-facial-P441325?icid2=products%20grid:p441325:product
Product URL: https://www.sephora.com/product/pore-purifying-black-strips-P288533?icid2=products%20grid:p288533:product
Product URL: https://www.sephora.com/product/photo-finish-primer-stick-P421233?icid2=products%20grid:p421233:product
Product URL: https://www.sephora.com/product/mission-perfection-serum-P399981?icid2=products%20grid:p399981:product
Product URL: https://www.sephora.com/product/line-smoother-oil-free-face-moisturizer-P12574?icid2=products%20grid:p12574:product
Product 

Product URL: https://www.sephora.com/product/capture-youth-redness-soother-age-delay-anti-redness-soothing-serum-P427730?icid2=products%20grid:p427730:product
Product URL: https://www.sephora.com/product/clarifying-spot-solution-P417941?icid2=products%20grid:p417941:product
Product URL: https://www.sephora.com/product/breakout-clearing-booster-P443364?icid2=products%20grid:p443364:product
Product URL: https://www.sephora.com/product/vitamin-b3-niacinamide-serum-rescue-troubled-skin-P411358?icid2=products%20grid:p411358:product
Product URL: https://www.sephora.com/product/collagen-serum-elasticity-suppleness-P391637?icid2=products%20grid:p391637:product
Product URL: https://www.sephora.com/product/silver-powder-P440499?icid2=products%20grid:p440499:product
Product URL: https://www.sephora.com/product/help-me-P4647?icid2=products%20grid:p4647:product
Product URL: https://www.sephora.com/product/rose-lifting-serum-P309212?icid2=products%20grid:p309212:product
Product URL: https://www.seph

Product URL: https://www.sephora.com/product/white-lucent-onmakeup-spot-correcting-serum-broad-spectrum-spf-25-P416541?icid2=products%20grid:p416541:product
Product URL: https://www.sephora.com/product/brightening-booster-drops-P442784?icid2=products%20grid:p442784:product
Product URL: https://www.sephora.com/product/charcoal-spot-corrector-P428818?icid2=products%20grid:p428818:product
Product URL: https://www.sephora.com/product/dynaspot-P381439?icid2=products%20grid:p381439:product
Product URL: https://www.sephora.com/product/maskimizer-tm-skin-optimizing-mask-primer-P405016?icid2=products%20grid:p405016:product
Product URL: https://www.sephora.com/product/kinetic-treatment-serum-P426072?icid2=products%20grid:p426072:product
Product URL: https://www.sephora.com/product/biorewind-am-full-spectrum-antioxidant-serum-P411500?icid2=products%20grid:p411500:product
Product URL: https://www.sephora.com/product/high-performance-vitamin-a-facial-treatment-P72705?icid2=products%20grid:p72705:pr

In [25]:
treatment_dict

{'brand': ['Clarins',
  'Drunk Elephant',
  'Drunk Elephant',
  'Drunk Elephant',
  'SUNDAY RILEY',
  'SK-II',
  'Estée Lauder',
  'OLEHENRIKSEN',
  'The Ordinary',
  'The Ordinary',
  'The Ordinary',
  'Drunk Elephant',
  'Peace Out',
  'Dr. Dennis Gross Skincare',
  'Glow Recipe',
  'The Ordinary',
  'Drunk Elephant',
  'The Ordinary',
  'La Mer',
  'Tatcha',
  'The Ordinary',
  "Kiehl's Since 1851",
  'Peace Out',
  'Estée Lauder',
  'La Mer',
  'Drunk Elephant',
  'Drunk Elephant',
  'GLAMGLOW',
  'The Ordinary',
  'Volition Beauty',
  'SUNDAY RILEY',
  'Lancôme',
  'Dr. Jart+',
  'SUNDAY RILEY',
  "Kiehl's Since 1851",
  'Caudalie',
  'Dr. Dennis Gross Skincare',
  'FARSÁLI',
  'The Ordinary',
  'The Ordinary',
  'The Ordinary',
  'Drunk Elephant',
  'Shiseido',
  'The Ordinary',
  'The Ordinary',
  'First Aid Beauty',
  'CLINIQUE',
  'Dr. Jart+',
  'SUNDAY RILEY',
  'Murad',
  'Biossance',
  'SUNDAY RILEY',
  'The Ordinary',
  'SUNDAY RILEY',
  'FARSÁLI',
  'Algenist',
  'Dr. Den

In [23]:
treatments_df = pd.DataFrame.from_dict(treatment_dict)

In [26]:
treatments_df.to_csv(r"/Users/florencialeoni/code/i_feel_pretty/data/treatments.csv")

In [36]:
wellness_url = product_url_finder("https://www.sephora.com/shop/wellness-skincare", 16)

Scraping: https://www.sephora.com/shop/wellness-skincare?pageSize=10&currentPage=1
Scraping: https://www.sephora.com/shop/wellness-skincare?pageSize=10&currentPage=2
Scraping: https://www.sephora.com/shop/wellness-skincare?pageSize=10&currentPage=3
Scraping: https://www.sephora.com/shop/wellness-skincare?pageSize=10&currentPage=4
Scraping: https://www.sephora.com/shop/wellness-skincare?pageSize=10&currentPage=5
Scraping: https://www.sephora.com/shop/wellness-skincare?pageSize=10&currentPage=6
Scraping: https://www.sephora.com/shop/wellness-skincare?pageSize=10&currentPage=7
Scraping: https://www.sephora.com/shop/wellness-skincare?pageSize=10&currentPage=8
Scraping: https://www.sephora.com/shop/wellness-skincare?pageSize=10&currentPage=9
Scraping: https://www.sephora.com/shop/wellness-skincare?pageSize=10&currentPage=10
Scraping: https://www.sephora.com/shop/wellness-skincare?pageSize=10&currentPage=11
Scraping: https://www.sephora.com/shop/wellness-skincare?pageSize=10&currentPage=12
S

In [37]:
wellness_series = pd.Series(wellness_url)

In [39]:
wellness_series.to_csv(r"/Users/florencialeoni/code/i_feel_pretty/data/wellness_urls.csv")

In [44]:
wellness_dict = product_info_finder(wellness_url)

Product URL: https://www.sephora.com/product/silk-pillowcase-standard-queen-P402944?icid2=products%20grid:p402944:product
Product URL: https://www.sephora.com/product/hair-sweet-hair-P415922?icid2=products%20grid:p415922:product
Product URL: https://www.sephora.com/product/jade-facial-roller-P436891?icid2=products%20grid:p436891:product
Product URL: https://www.sephora.com/product/silk-pillowcase-king-P402943?icid2=products%20grid:p402943:product
Product URL: https://www.sephora.com/product/collagen-peptides-P433638?icid2=products%20grid:p433638:product
Product URL: https://www.sephora.com/product/sephora-collection-x-olly-P441799?icid2=products%20grid:p441799:product
Product URL: https://www.sephora.com/product/contouring-de-puffing-rose-quartz-roller-P438628?icid2=products%20grid:p438628:product
Product URL: https://www.sephora.com/product/skin-heroes-pre-probiotic-clear-skin-supplement-P443349?icid2=products%20grid:p443349:product
Product URL: https://www.sephora.com/product/daily-c

Product URL: https://www.sephora.com/product/beauty-shroom-vegan-collagen-protection-P436087?icid2=products%20grid:p436087:product
Product URL: https://www.sephora.com/product/killer-nails-supplements-P386761?icid2=products%20grid:p386761:product
Product URL: https://www.sephora.com/product/collagen-pop-P432259?icid2=products%20grid:p432259:product
Product URL: https://www.sephora.com/product/noni-glow-skinfood-dietary-supplement-powder-P418824?icid2=products%20grid:p418824:product
Product URL: https://www.sephora.com/product/coconut-oil-pullers-P429519?icid2=products%20grid:p429519:product
Product URL: https://www.sephora.com/product/health-weight-management-dietary-supplements-P232600?icid2=products%20grid:p232600:product
Product URL: https://www.sephora.com/product/cacao-golde-P443817?icid2=products%20grid:p443817:product
Product URL: https://www.sephora.com/product/original-golde-P443815?icid2=products%20grid:p443815:product
Product URL: https://www.sephora.com/product/hair-supplem

Product URL: https://www.sephora.com/product/collagen-beauty-greens-P434357?icid2=products%20grid:p434357:product
Product URL: https://www.sephora.com/product/mini-caddy-trio-P440049?icid2=products%20grid:p440049:product
Product URL: https://www.sephora.com/product/beauty-dust-box-P427534?icid2=products%20grid:p427534:product
Product URL: https://www.sephora.com/product/hair-supplement-for-thinning-hair-P409847?icid2=products%20grid:p409847:product
Product URL: https://www.sephora.com/product/golden-latte-mushroom-mix-with-shiitake-turmeric-P434351?icid2=products%20grid:p434351:product
Product URL: https://www.sephora.com/product/chai-latte-mushroom-mix-with-turkey-tail-reishi-P434354?icid2=products%20grid:p434354:product
Product URL: https://www.sephora.com/product/spiked-lala-glass-water-bottle-P422320?icid2=products%20grid:p422320:product
Product URL: https://www.sephora.com/product/beauty-collagen-lavender-lemon-P436649?icid2=products%20grid:p436649:product
Product URL: https://www

In [47]:
wellness_df = pd.DataFrame.from_dict(wellness_dict)

In [50]:
wellness_df

,brand,name,price,n_of_reviews,n_of_loves,review_score,size,clean_product,category
0,Slip,Silk Pillowcase - Standard/Queen,85.00,222,56021,4.2658,,0,None
1,HUM Nutrition,Hair Sweet Hair Growth - Vegan Gummies,25.00,195,21267,4.0872,60 Vegan berry gummy hearts•,0,None
2,Herbivore,Jade Facial Roller,30.00,95,12405,3.8842,None,1,Tools
3,Slip,Silk Pillowcase - King,105.00,48,12033,4.4792,,0,None
4,Vital Proteins,Collagen Peptides,25.00,63,6333,4.2222,10 oz,0,Beauty Supplements
5,SEPHORA COLLECTION,Sephora Collection x OLLY,15.00,11,0,3.9091,50 Capsules•,0,Beauty Supplements
6,BeautyBio,Contouring + De-Puffing Rose Quartz Roller,60.00,17,5316,4.8235,None,0,None
7,HUM Nutrition,Skin Heroes Pre + Probiotic Clear Skin Supplement,40.00,1,2461,5.0000,60 Capsules•,0,None
8,HUM Nutrition,Daily Cleanse Clear Skin and Body Detox Supple...,25.00,332,30843,4.0030,60 Capsules•,0,None
9,Moon Juice,SuperYou® Daily Stress Management,49.00,67,7542,4.5672,60 capsules•,0,Beauty Supplements


In [49]:
wellness_df.to_csv(r"/Users/florencialeoni/code/i_feel_pretty/data/wellness.csv")

In [52]:
eyecare_url = product_url_finder("https://www.sephora.com/shop/eye-treatment-dark-circle-treatment", 23)

Scraping: https://www.sephora.com/shop/eye-treatment-dark-circle-treatment?pageSize=10&currentPage=1
Scraping: https://www.sephora.com/shop/eye-treatment-dark-circle-treatment?pageSize=10&currentPage=2
Scraping: https://www.sephora.com/shop/eye-treatment-dark-circle-treatment?pageSize=10&currentPage=3
Scraping: https://www.sephora.com/shop/eye-treatment-dark-circle-treatment?pageSize=10&currentPage=4
Scraping: https://www.sephora.com/shop/eye-treatment-dark-circle-treatment?pageSize=10&currentPage=5
Scraping: https://www.sephora.com/shop/eye-treatment-dark-circle-treatment?pageSize=10&currentPage=6
Scraping: https://www.sephora.com/shop/eye-treatment-dark-circle-treatment?pageSize=10&currentPage=7
Scraping: https://www.sephora.com/shop/eye-treatment-dark-circle-treatment?pageSize=10&currentPage=8
Scraping: https://www.sephora.com/shop/eye-treatment-dark-circle-treatment?pageSize=10&currentPage=9
Scraping: https://www.sephora.com/shop/eye-treatment-dark-circle-treatment?pageSize=10&curr

In [53]:
eyecare_series = pd.Series(eyecare_url)

In [54]:
eyecare_series.to_csv(r"/Users/florencialeoni/code/i_feel_pretty/data/eyecare_url.csv")

In [55]:
eyecare_dict = product_info_finder(eyecare_url)

Product URL: https://www.sephora.com/product/banana-bright-eye-creme-P426339?icid2=products%20grid:p426339:product
Product URL: https://www.sephora.com/product/c-tango-multivitamin-eye-cream-P429515?icid2=products%20grid:p429515:product
Product URL: https://www.sephora.com/product/benefiance-wrinkleresist24-intensive-eye-contour-cream-P173620?icid2=products%20grid:p173620:product
Product URL: https://www.sephora.com/product/the-eye-concentrate-P416343?icid2=products%20grid:p416343:product
Product URL: https://www.sephora.com/product/advanced-night-repair-eye-supercharged-complex-P433885?icid2=products%20grid:p433885:product
Product URL: https://www.sephora.com/product/shaba-complex-eye-serum-P419223?icid2=products%20grid:p419223:product
Product URL: https://www.sephora.com/product/creamy-eye-treatment-with-avocado-P422000?icid2=products%20grid:p422000:product
Product URL: https://www.sephora.com/product/moisturizing-eye-bomb-P422905?icid2=products%20grid:p422905:product
Product URL: ht

Product URL: https://www.sephora.com/product/re-storative-eye-creme-P386739?icid2=products%20grid:p386739:product
Product URL: https://www.sephora.com/product/uplifting-transformation-eye-creme-P399110?icid2=products%20grid:p399110:product
Product URL: https://www.sephora.com/product/pack-your-bags-911-undereye-rescue-patches-P409630?icid2=products%20grid:p409630:product
Product URL: https://www.sephora.com/product/kale-aloe-sunflower-oil-tripeptide-5-age-prevention-superfood-eye-cream-P416923?icid2=products%20grid:p416923:product
Product URL: https://www.sephora.com/product/cold-plasma-eye-P426831?icid2=products%20grid:p426831:product
Product URL: https://www.sephora.com/product/renergie-lift-multi-action-eye-P376136?icid2=products%20grid:p376136:product
Product URL: https://www.sephora.com/product/potent-c-tm-power-eye-cream-P433456?icid2=products%20grid:p433456:product
Product URL: https://www.sephora.com/product/all-about-eyes-serum-de-puffing-eye-massage-P257537?icid2=products%20g

Product URL: https://www.sephora.com/product/essential-fx-acyl-glutathionesmoothing-brightening-under-eye-cream-P433954?icid2=products%20grid:p433954:product
Product URL: https://www.sephora.com/product/24-7-retinol-eye-cream-P417625?icid2=products%20grid:p417625:product
Product URL: https://www.sephora.com/product/goat-milk-de-puffing-eye-balm-P407401?icid2=products%20grid:p407401:product
Product URL: https://www.sephora.com/product/hydro-dynamic-ultimate-moisture-for-eyes-P293206?icid2=products%20grid:p293206:product
Product URL: https://www.sephora.com/product/redness-recovery-antioxidant-eye-treatment-P442849?icid2=products%20grid:p442849:product
Product URL: https://www.sephora.com/product/youth-dose-eye-treatment-P435211?icid2=products%20grid:p435211:product
Product URL: https://www.sephora.com/product/age-reversal-eye-complex-P423145?icid2=products%20grid:p423145:product
Product URL: https://www.sephora.com/product/sonic-awakening-eye-massager-P435206?icid2=products%20grid:p4352

Product URL: https://www.sephora.com/product/immortelle-eye-balm-P39912?icid2=products%20grid:p39912:product
Product URL: https://www.sephora.com/product/energie-de-vie-illuminating-anti-fatigue-cooling-eye-gel-P419634?icid2=products%20grid:p419634:product
Product URL: https://www.sephora.com/product/hello-fab-kona-eye-stick-P434552?icid2=products%20grid:p434552:product
Product URL: https://www.sephora.com/product/wrinkle-release-eye-cream-with-peptides-P416590?icid2=products%20grid:p416590:product
Product URL: https://www.sephora.com/product/divine-eyes-P396251?icid2=products%20grid:p396251:product
Product URL: https://www.sephora.com/product/liquid-gold-firming-eye-cream-P442537?icid2=products%20grid:p442537:product
Product URL: https://www.sephora.com/product/eye-contour-gel-P94820?icid2=products%20grid:p94820:product
Product URL: https://www.sephora.com/product/powerful-strength-line-reducing-eye-brightening-concentrate-P427625?icid2=products%20grid:p427625:product
Product URL: htt

In [56]:
eyecare_df = pd.DataFrame.from_dict(eyecare_dict)

In [57]:
eyecare_df.to_csv(r"/Users/florencialeoni/code/i_feel_pretty/data/eyecare.csv")

In [59]:
facetreatments_df = pd.read_excel("/Users/florencialeoni/code/i_feel_pretty/data/treatments.xls")

In [60]:
facetreatments_df

,brand,name,price,n_of_reviews,n_of_loves,review_score,size,clean_product,category
0,Clarins,Double Serum Complete Age Control Concentrate,89,391,16293,4.0997,1 oz,0,Face Serums
1,Drunk Elephant,C-Firma Vitamin C Day Serum,80,1000,135964,4.1097,1 oz,1,Face Serums
2,Drunk Elephant,TLC Sukari Babyfacial Mask,80,1000,178783,4.4283,1.69 oz,1,Face Masks
3,Drunk Elephant,T.L.C. Framboos‚Ñ¢ Glycolic Night Serum,90,1000,144421,4.2968,1 oz,1,Face Serums
4,SUNDAY RILEY,Good Genes All-In-One Lactic Acid Treatment,158,2000,154923,4.3046,1.7 oz,0,Face Serums
5,SK-II,Facial Treatment Essence,179,788,68880,4.0622,None,0,Mists & Essences
6,Est√©e Lauder,Advanced Night Repair Synchronized Recovery Co...,168,11,12952,5.0000,None,0,Face Serums
7,OLEHENRIKSEN,Truth Serum¬Æ,49,4000,171521,4.2454,1 oz,0,Face Serums
8,The Ordinary,AHA 30% + BHA 2% Peeling Solution,7.2,210,38876,4.8000,1 oz,0,Facial Peels
9,The Ordinary,"""Buffet""",14.8,468,78360,4.1581,1 oz,0,Face Serums


In [61]:
# Function to create a new SQLite3 Database
def create_connection(db_file):
    try:
        conn = sqlite3.connect(db_file)
    except Error as e:
        print(e)
    finally:
        conn.close()

In [62]:
# Use create_connection function to create Database
create_connection("/Users/florencialeoni/code/i_feel_pretty/sephora.db")

In [63]:
# Create a connection object using the connect function 
# To establish a connection with "sephora.db"
connection = sqlite3.connect("sephora.db")
# Create a cursor object by calling the cursor method
cursor = connection.cursor()

In [67]:
skincare_df.to_sql("moisturizers", con = connection)

In [68]:
treatments_df.to_sql("treatments", con = connection)

In [69]:
wellness_df.to_sql("wellness", con = connection)

In [70]:
eyecare_df.to_sql("eyecare", con = connection)